# Домашнее задание 5.

Ниже приведено решение задачи Walmart TripType prediction.

Сначала мы строим простую baseline модель используя алгоритм случайного леса, затем делаем Exploratory Data Analysis, и наконец генерим новый набор признаков и на нем обучаем алгоритм градиентного бустинга.

Разберитесь в этом нотбуке: 
- в чем заключается задача?
- какие признаки мы генерим?

Существует 3 популярные библиотеки реализующие алгоритм градиентного бустинга:
- xgboost https://xgboost.readthedocs.io/en/stable/python/python_intro.html
- catboost https://catboost.ai/en/docs/concepts/python-quickstart
- lightgbm https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

 
Решение отправляйте на почту kurmukovai@gmail.com, до полуночи 24 марта, 23:59:59 с темой письма [iitp-intro-ds-2024-ha5-Surname], например [iitp-intro-ds-2024-ha5-Kurmukov]

* если вам нужно будет обработать пропуски (missing values) обрабатывайте их любым разумным образом (см. семинар 3)
* все 3 библиотеки умеют работать с категориальным переменными "из коробки":
- https://xgboost.readthedocs.io/en/stable/tutorials/categorical.html
- https://lightgbm.readthedocs.io/en/latest/Advanced-Topics.html

возможно необходимо строковые признаки закодировать в числа 1..n .

# п. 1-5 смотрите ниже


Данные без пароля можно скачать тут https://disk.yandex.ru/d/xTXpgXJwiWyYSQ

Ниже предложено два baseline решения, изучите их прежде чем переходить к разделам 6-7.


# 6. Вопросы про данные

В вопросах про данные используйте выборку `train.csv`

1. Сколько всего уникальных типов покупок (TripType) в данной задаче?
2. В какой день недели совершается больше всего покупок?
3. Какой мультиклассовой точности соответствует логлосс 0.997 в данной задаче?
4. Опишите качественно самый частый и самый редкий типы покупок в данных: в какой день они происходят, каким категорям товаров соответствуют, какой в них средний размер чека (количество позиций) и т.д.


# 7. Compare `xgboost`, `catboost` and `lightgbm`

Ваша задача состоит в том чтобы познакомиться с 3 имплементациями градиентного бустинга. Перед вами нет задачи выбить "наилучшее" качество. 

1. Отредактируйте п.4. Разделите данные `train.csv` на тренировочную и тестовую выборки в отношении 8 к 2.

2. Запустите модели `xgboost`, `lightgbm` и `catboost` с гиперпараметрами предложенными выше для `catboost`:
```
ctb_params = {
    'depth': 4,
    'learning_rate': .3,
    'loss_function': 'MultiClass',
    'iterations': 100
}
```
остальные гиперпараметры моделей используйте "по умолчанию". Сравните качество полученных моделей на "внутренней" тестовой выборке из вопроса 1 в терминах логистического лосса и в терминах мультиклассовой точности (multi-class accuracy).

3. Перечислите какие гиперпараметры для регуляризации существуют в трех имплементациях градиентного бустинга. Какие из них повторяются, а какие отличаются?

4. Повторите п.2, но теперь в качестве гиперпараметров в моделях используйте параметры по-умолчанию из `lightgbm`:
`num_leaves`, `learning_rate`, `n_estimators`. Сравните результаты с результатами из вопроса 2. 

5. (по возможности) сделайте предсказание лучшей из моделей на тестовой выборке (test.csv) для которой у вас нет столбца `TripType`, засабмитьте результат на kaggle и сравните оценки logloss между внутренним и отложенным тестом, прокомментируйте результат.


# Walmart TripType prediction

https://www.kaggle.com/c/walmart-recruiting-trip-type-classification

>For this competition, you are tasked with categorizing shopping trip types based on the items that customers purchased. To give a few hypothetical examples of trip types: a customer may make a small daily dinner trip, a weekly large grocery trip, a trip to buy gifts for an upcoming holiday, or a seasonal trip to buy clothes.


## Multi-class classification, goal is to predict `type of the trip`.

In [ ]:
# !kaggle competitions download -c walmart-recruiting-trip-type-classification

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv('train.csv')
# test = pd.read_csv('walmart-recruiting-trip-type-classification/test.csv')

In [ ]:
train.head()

### Columns description

- `TripType` - a categorical id representing the type of shopping trip the customer made. This is the ground truth that you are predicting. TripType_999 is an "other" category.
- `VisitNumber` - an id corresponding to a single trip by a single customer
- `Weekday` - the weekday of the trip
- `Upc`- the UPC number of the product purchased
- `ScanCount` - the number of the given item that was purchased. A negative value indicates a product return.
- `DepartmentDescription` - a high-level description of the item's department
- `FinelineNumber` - a more refined category for each of the products, created by Walmart


In [ ]:
train.shape

In [ ]:
train.count()

In [ ]:
train.TripType.value_counts()

# 1. Understand the task.

> Every single ml task has its features, and there are no universal solutions, only generally working principles.

We need to predict type of the visit: `TripType`. Every row of the data table contains information about a single product not a visit, threfore we need to combine the information about the visit from all the purchases. 

![](trip-type.png)

Unfortunately, we do not have the information about customers, so we do not know if some of the Visits were performed by the same person.


In [ ]:
train.head(5)

# 2. Build a simple baseline model

- Count number of purchases in a Visit
- Create binary column `is_weekend`
- Drop all the remaining columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
def is_weekend(day):
    return int(day in ['Saturday', 'Sunday'])

df_train = train.copy()

# Generate is_weekend
df_train['is_weekend'] = df_train.Weekday.apply(is_weekend)

# Generate n_products
gp_n_products = df_train.groupby('VisitNumber')['ScanCount'].count()
df_train['n_products'] = df_train.VisitNumber.map(gp_n_products)

# drop duplicated Visit numbers
df_train = df_train.drop_duplicates(subset=['VisitNumber']).reset_index(drop=True)

# drop all columns except `is_weekend`, `n_products` and `TripType`
df_train = df_train.drop(['VisitNumber', 'Weekday', 'Upc', 'ScanCount',
                          'DepartmentDescription', 'FinelineNumber'], axis=1)

# Encode TripType so unique values are from 0 to (m-1), where m is number of classes
encoder = LabelEncoder().fit(df_train['TripType'])
df_train['TripType_encoded'] =  encoder.transform(df_train['TripType'])
df_train = df_train.drop('TripType', axis=1)

# Create separate variables X and y
X = df_train.drop('TripType_encoded', axis=1)
y = df_train.TripType_encoded

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split data into train and test, use parameter `stratify=y`

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=774, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [ ]:
# train a RandomForest model with default hyperparameters

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

---
For this tash we are using a log loss $$- y \log p - (1-y)\log(1-p)$$
in the following multi class form:

$$\frac{1}{N}\sum_{i=1}^N \log {\left(\frac{e^{a_{it_i}}}{\sum_{j=0}^{M-1} e^{a_{ij}}}\right)}$$

$t \in \{0\ldots M-1\}$, $M$ is number of classes, $N$ is number of objects. Numerator is $a_{it_{i}}$ = \[ unnormalized probability of an $i$'th object to be assigned to the right class $t_i$\], thus:

$$p_{it_i} = \frac{e^{a_{it_i}}}{\sum_{j=0}^{M-1} e^{a_{ij}}}$$

see for example https://towardsdatascience.com/cross-entropy-for-classification-d98e7f974451

In [ ]:
# predict y_test and compute multi-class log loss of your prediction

y_pred = clf.predict_proba(X_test)
log_loss(y_test, y_pred)

In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# just to get an intuition on whether it is a good a bad prediction compute an accuracy of your model

y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
# Remember this is a 38 class classification, count predicted classes

np.unique(y_pred, return_counts=True)

In [ ]:
# Check value counts of a TripType on a train set

y_train.value_counts()

## Compare with the constant prediction


In [ ]:
# Compute a log loss and an accuracy of a constant prediction (predict most frequent TripType)

y_dummy = [5]*19135
print(accuracy_score(y_test, y_dummy))

y_dummy_proba = np.zeros((19135, 38))
y_dummy_proba[:, 5] = 1
print(log_loss(y_test, y_dummy_proba))

## Conclusions on the baseline
1. Even with this simple features and relatively stupid predictions we are better than a constant prediction.
2. Classifier mostly predicts frequent classes.
3. Some frequent classes are predicted and some are not. This may be due to the fact that predicted
classes are better described by the generated features.

# 3. Deeper look on the features.

### Columns description

- `TripType` - a categorical id representing the type of shopping trip the customer made. This is the ground truth that you are predicting. TripType_999 is an "other" category.
- `VisitNumber` - an id corresponding to a single trip by a single customer
- `Weekday` - the weekday of the trip
- `Upc`- the UPC number of the product purchased
- `ScanCount` - the number of the given item that was purchased. A negative value indicates a product return.
- `DepartmentDescription` - a high-level description of the item's department
- `FinelineNumber` - a more refined category for each of the products, created by Walmart


In [ ]:
train.head()

In [ ]:
train.dtypes

All features, except `ScanCount` are categorical, a negative value of a `ScanCount` indicates a product return.

### 3.1 VisitNumber 
is an indicator of a visit, we need it to aggregate different purchases,
but the number itself is not important it is just an index. Let's have a quick look on it.

In [ ]:
train.VisitNumber.value_counts()

In [ ]:
sns.histplot(data=train.VisitNumber.value_counts(), bins=50);

In [ ]:
train.VisitNumber.value_counts().value_counts().sort_index()[:20]

--- 

- More than half of all Visit consists of 4 or less purchases
- 90% of Visits consist of 17 or less purchases

### 3.2 Weekday

In [ ]:
# How many visits are on different weekdays
weekdays = ['Monday', 'Tuesday', 'Wednesday','Thursday','Friday', 'Saturday', 'Sunday']

train.Weekday.value_counts().reindex(weekdays)

In [ ]:
# Do we have more returns on some weekdays?

train['IsReturn'] = train['ScanCount'].apply(lambda x: x < 0)
train.groupby(['Weekday'])['IsReturn'].sum().sort_values().reindex(weekdays)

In [ ]:
# Is there only `-1` returns?

train[train.IsReturn].ScanCount.value_counts()

### 3.3 DepartmentDescription

In [ ]:
# What are the most popular Departments? (total sum over ScanCount)

train.groupby('DepartmentDescription').ScanCount.sum().sort_values(ascending=False)[:20]

In [ ]:
# Does most popular DepartmentDescription differ for different TripTypes?
# > Allows to more or less deanonymize `TripType`.

gp = train.groupby('TripType')['DepartmentDescription'].value_counts().reset_index(name='Count')
gp2 = gp.groupby(['TripType'])[['Count','DepartmentDescription']]\
                                                           .apply(pd.DataFrame.nlargest, n=3, columns=['Count'])\
                                                           .reset_index()\
                                                           .drop('level_1', axis=1)

gp2[:15]

In [ ]:
# From our baseline, recall most popular TripTypes, let's deanonymize them.
# Select subset of the previous table with TripType in [40, 39, 37, 38, 25]

gp2[gp2.TripType.isin([40, 39, 37, 38, 25])]

In [ ]:
# What are the DepartmentDescription with most returns?

train.groupby('DepartmentDescription')['IsReturn'].sum().sort_values(ascending=False)[:15]

### 3.4 FinelineNumber

> according to the data description `FinelineNumber` is just a more detailed `DepartmentDescription`

In [ ]:
# Build a crosstab between DepartmentDescription and FinelineNumber,
# for most popular DepartmentDescription (total ScanCount > 20_000)
# and most popular FinelineNumber (total ScanCount > 2000)


popular_dd = train.groupby('DepartmentDescription').ScanCount.sum().sort_values(ascending=False)[:10].index
sub = train[train.DepartmentDescription.isin(popular_dd)]
sub = sub[sub.FinelineNumber.isin(sub.FinelineNumber.value_counts().iloc[:15].index)]

tab = pd.crosstab(sub.FinelineNumber, sub.DepartmentDescription)
tab

### 3.5 Upc

In [ ]:
train.Upc.value_counts()

## Conclusions on EDA

 - Visits are mostly consist of small number of products
 - Large visits are on weekends
 - `TripType` depends on `DepartmentDescription` and `FinelineNumber`
 - `Upc` is something like a bar code, could be usefull but contains almost 100k unique values
 

# 4. Generate features

In [ ]:
train = pd.read_csv('train.csv')
# test = pd.read_csv('walmart-recruiting-trip-type-classification/test.csv')

In [ ]:
def return_nth(x, n=0):
    try:
        return x[n]
    except:
        return np.nan
    
def return_nth_val(x, n=0):
    try:
        return x[n]
    except:
        return 0
    
def is_weekend(x):
    return int(x in ['Sunday', 'Saturday'])

def has_return(x):
    return int(any(_x < 0 for _x in x))

def sum_return(x):
    return np.sum([_x for _x in x if _x < 0])

In [ ]:
df = train.copy()

In [ ]:
def generate_features(df):
    
    df = df.copy()
    
    # 1. Most frequent DepartmentDescription in the purchase, second most frequent, third, fourth.
    f = lambda grp: grp.value_counts().nlargest(6)
    x = df.groupby('VisitNumber')['DepartmentDescription'].apply(f).reset_index()
    gp = x.groupby('VisitNumber')['level_1'].unique()

    df['PopularCategory'] = df.VisitNumber.map(gp)
    for i in range(4):
        df[f'Category_{i}'] = df['PopularCategory'].apply(return_nth, args=[i])

    # 2. Same for FinelineNumber, but 6
    x = df.groupby('VisitNumber')['FinelineNumber'].apply(f).reset_index()
    gp = x.groupby('VisitNumber')['level_1'].unique()

    df['PopularFineline'] = df.VisitNumber.map(gp)
    for i in range(6):
        df[f'Fineline_{i}'] = df['PopularFineline'].apply(return_nth, args=[i]).astype(object)

    # 3. Count number of unique DepartmentDescription in the purchase
    gp = df.groupby('VisitNumber')['DepartmentDescription'].nunique()
    df['#Unique_Department'] = df.VisitNumber.map(gp)

    # 4. Count number of unique FinelineNumber in the purchase.
    gp = df.groupby('VisitNumber')['FinelineNumber'].nunique()
    df['#Unique_Fineline'] = df.VisitNumber.map(gp)

    # 5. Count ScanCount (number of unique Upc in the purchase)

    gp = df.groupby('VisitNumber')['ScanCount'].count()
    df['#UniqueScanCount'] = df['VisitNumber'].map(gp)   

    # 6. Sum ScanCount

    gp = df.groupby('VisitNumber')['ScanCount'].sum()
    df['TotalScanCount'] = df['VisitNumber'].map(gp)

    # 7. Is weekend

    df['is_Weekend'] = df['Weekday'].apply(is_weekend).astype(object)

    # 8. Sum returns

    gp = df.groupby('VisitNumber')['ScanCount'].apply(sum_return)
    df['total_return'] = df.VisitNumber.map(gp)
    df['total_return'].fillna(0, inplace=True)

    # Drop old and intermediate features

    df = df.drop(['Upc', 'ScanCount', 'DepartmentDescription',
                  'FinelineNumber', 'PopularFineline', 'PopularCategory'], axis=1)

    # Drop duplicated rows (with the same VisitNumber)

    df = df.drop_duplicates(keep='first')
    
    return df

In [ ]:
# Running this cell takes about 10 minutes
df = generate_features(df)

# 5. Train a model

In [ ]:
# !pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
for i, col in enumerate(df.columns):
    print(i-2, col, df[col].dtype)

In [ ]:
for col in df.columns:
    if col.startswith('Category'):
        df[col]=df[col].apply(str)
    if col.startswith('Fineline'):
        df[col]=df[col].apply(str)

In [ ]:
train.fillna('NaN', inplace=True)
x_train, x_eval = train_test_split(df, test_size=0.1, random_state=10, shuffle=True)

cat_features = [*list(range(10)), 15]

data_train = Pool(x_train.drop(['TripType', 'VisitNumber'], axis=1), 
                  label=x_train.TripType,
                  cat_features=cat_features)

data_eval = Pool(x_eval.drop(['TripType', 'VisitNumber'], axis=1), 
                 label=x_eval.TripType,
                 cat_features=cat_features)

In [ ]:
ctb_params = {
    'depth': 4,
    'learning_rate': .3,
    'l2_leaf_reg': 3,
    'loss_function': 'MultiClass',
    'verbose': 1,
    'thread_count': 12,
    'iterations': 100
}

In [ ]:
model = CatBoostClassifier(**ctb_params)
model.fit(X = data_train, silent=False, eval_set=data_eval)

In [ ]:
# model.save_model('catboost_depth=4_lr=.3_l2=3')
# model = CatBoostClassifier()